In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import time
import argparse


class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.cn1 = nn.Conv2d(1, 16, 3, 1)
        self.cn2 = nn.Conv2d(16, 32, 3, 1)
        self.dp1 = nn.Dropout2d(0.10)
        self.dp2 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(4608, 64) # 4608 is basically 12 X 12 X 32
        self.fc2 = nn.Linear(64, 10)
 
    def forward(self, x):
        x = self.cn1(x)
        x = F.relu(x)
        x = self.cn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dp1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp2(x)
        x = self.fc2(x)
        op = F.log_softmax(x, dim=1)
        return op
    
    
def train(args):
    torch.manual_seed(0)
    device = torch.device("cpu")
    train_dataloader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1302,), (0.3069,))])),
        batch_size=128, shuffle=True)  
    model = ConvNet()
    optimizer = optim.Adadelta(model.parameters(), lr=0.5)
    model.train()
    for epoch in range(args.epochs):
        for b_i, (X, y) in enumerate(train_dataloader):
            X, y = X.to(device), y.to(device)
            pred_prob = model(X)
            loss = F.nll_loss(pred_prob, y) # nll is the negative likelihood loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if b_i % 10 == 0:
                print('epoch: {} [{}/{} ({:.0f}%)]\t training loss: {:.6f}'.format(
                    epoch, b_i, len(train_dataloader),
                    100. * b_i / len(train_dataloader), loss.item()))

In [ ]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--epochs', default=1, type=int)
    args = parser.parse_args(args={})
    start = time.time()
    train(args)
    print(f"Finished training in {time.time()-start} secs")

In [ ]:
main()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode

epoch: 0 [0/469 (0%)]	 training loss: 2.314504
epoch: 0 [10/469 (2%)]	 training loss: 1.530618
epoch: 0 [20/469 (4%)]	 training loss: 0.911551
epoch: 0 [30/469 (6%)]	 training loss: 0.578104
epoch: 0 [40/469 (9%)]	 training loss: 0.356701
epoch: 0 [50/469 (11%)]	 training loss: 0.585043
epoch: 0 [60/469 (13%)]	 training loss: 0.264025
epoch: 0 [70/469 (15%)]	 training loss: 0.226957
epoch: 0 [80/469 (17%)]	 training loss: 0.217207
epoch: 0 [90/469 (19%)]	 training loss: 0.251942
epoch: 0 [100/469 (21%)]	 training loss: 0.145386
epoch: 0 [110/469 (23%)]	 training loss: 0.301795
epoch: 0 [120/469 (26%)]	 training loss: 0.290040
epoch: 0 [130/469 (28%)]	 training loss: 0.168344
epoch: 0 [140/469 (30%)]	 training loss: 0.181666
epoch: 0 [150/469 (32%)]	 training loss: 0.127011
epoch: 0 [160/469 (34%)]	 training loss: 0.088400
epoch: 0 [170/469 (36%)]	 training loss: 0.174289
epoch: 0 [180/469 (38%)]	 training loss: 0.094773
epoch: 0 [190/469 (41%)]	 training loss: 0.102140
epoch: 0 [200/46

It took roughly 50 seconds to train for 1 epoch, which equates to 469 batches, each of which has 128 data points. The only exception is the last batch, which has 32 fewer data points than usual (as there are 60,000 data points in total).

At this point, it is important to know what kind of machine this model is being trained on so that we know the reference context

In [ ]:
# !sudo apt-get install inxi
!inxi -F

System:    Host: 09b441dfb569 Kernel: 5.4.104+ x86_64 bits: 64 Console: tty 0
           Distro: Ubuntu 18.04.5 LTS
Machine:   Device: docker Mobo: Google model: Google Compute Engine serial: Board-GoogleCloud-E564885479FEEC526038172E5700BA79
           BIOS: Google v: Google date: 01/01/2011
Battery    Using dmidecode: unknown error occurred
CPU:       Single core Intel Xeon (-MT-) cache: 56320 KB
           clock speeds: max: 2200 MHz 1: 2200 MHz 2: 2200 MHz
Graphics:  Card: Failed to Detect Video Card!
           Display Server: X.org 1.20.8 driver: N/A
           tty size: 0x0 Advanced Data: N/A for root out of X
Network:   Card: Red Hat Virtio network device driver: virtio-pci
           IF: N/A state: N/A speed: N/A duplex: N/A mac: N/A
Drives:    HDD Total Size: 85.9GB (51.2% used)
           ID-1: /dev/sda model: PersistentDisk size: 85.9GB
Partition: ID-1: / size: 108G used: 37G (35%) fs: overlay dev: N/A
RAID:      No RAID devices: /proc/mdstat, md_mod kernel module present
S

While torch.multiprocessing helps spawn multiple Python processes within a machine (typically, we may spawn as many processes as there are CPU cores in the machine), torch.distributed enables communications between different machines as they work together to train the model. During execution, we need to explicitly launch our model training script from within each of these machines.

One of the built-in PyTorch communication backends, such as Gloo, will then take care of the communication between these machines. Inside each machine, multiprocessing will take care of parallelizing the training task across several processes.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import torch.multiprocessing as mp
import torch.distributed as dist

import os
import time
import argparse


class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.cn1 = nn.Conv2d(1, 16, 3, 1)
        self.cn2 = nn.Conv2d(16, 32, 3, 1)
        self.dp1 = nn.Dropout2d(0.10)
        self.dp2 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(4608, 64) # 4608 is basically 12 X 12 X 32
        self.fc2 = nn.Linear(64, 10)
 
    def forward(self, x):
        x = self.cn1(x)
        x = F.relu(x)
        x = self.cn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dp1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp2(x)
        x = self.fc2(x)
        op = F.log_softmax(x, dim=1)
        return op
     

def train(cpu_num, args):
    rank = args.machine_id * args.num_processes + cpu_num                        
    dist.init_process_group(                                   
    backend='gloo',                                         
    init_method='env://',                                   
    world_size=args.world_size,                              
    rank=rank                                               
    ) 
    torch.manual_seed(0)
    device = torch.device("cpu")
    train_dataset = datasets.MNIST('../data', train=True, download=True,
                                   transform=transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1302,), (0.3069,))]))  
    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset,
        num_replicas=args.world_size,
        rank=rank
    )
    train_dataloader = torch.utils.data.DataLoader(
       dataset=train_dataset,
       batch_size=args.batch_size,
       shuffle=False,            
       num_workers=0,
       sampler=train_sampler)
    model = ConvNet()
    optimizer = optim.Adadelta(model.parameters(), lr=0.5)
    model = nn.parallel.DistributedDataParallel(model)
    model.train()
    for epoch in range(args.epochs):
        for b_i, (X, y) in enumerate(train_dataloader):
            X, y = X.to(device), y.to(device)
            pred_prob = model(X)
            loss = F.nll_loss(pred_prob, y) # nll is the negative likelihood loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if b_i % 10 == 0 and cpu_num==0:
                print('epoch: {} [{}/{} ({:.0f}%)]\t training loss: {:.6f}'.format(
                    epoch, b_i, len(train_dataloader),
                    100. * b_i / len(train_dataloader), loss.item()))
         
            
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--num-machines', default=1, type=int,)
    parser.add_argument('--num-processes', default=1, type=int)
    parser.add_argument('--machine-id', default=0, type=int)
    parser.add_argument('--epochs', default=1, type=int)
    parser.add_argument('--batch-size', default=128, type=int)
    args = parser.parse_args(args={})
    
    args.world_size = args.num_processes * args.num_machines                
    os.environ['MASTER_ADDR'] = '127.0.0.1'              
    os.environ['MASTER_PORT'] = '8892'      
    start = time.time()
    mp.spawn(train, nprocs=args.num_processes, args=(args,))
    print(f"Finished training in {time.time()-start} secs")

In [ ]:
main()

ProcessExitedException: ignored